In [6]:
import copy

def manhattan_distance(state, goal):
    # map each tile in goal to its (i,j) position
    goal_pos = {}
    for i in range(3):
        for j in range(3):
            goal_pos[goal[i][j]] = (i, j)

    dist = 0
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile == '*':
                continue
            gi, gj = goal_pos[tile] # store the indexes of the tile in the goal state
            dist += abs(i - gi) + abs(j - gj)
    return dist


class Node:
    def __init__(self, state, g=0, h=0, parent=None):
        self.state = state      # puzzle configuration
        self.g = g              # cost so far
        self.h = h              # heuristic cost
        self.f = g + h          # f = g + h (A* score) [total cost]
        self.parent = parent    # reference to previous node

    def find(self):
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == '*':
                    return i, j

    def move(self, x0, y0, x1, y1):
        if 0 <= x1 <= 2 and 0 <= y1 <= 2:
            tempstate = copy.deepcopy(self.state)
            tempstate[x0][y0], tempstate[x1][y1] = tempstate[x1][y1], tempstate[x0][y0]
            return tempstate
        return None

    # NOTE: generate returns children with g incremented; h will be computed later
    def generate(self, movement):
        x, y = self.find()
        if movement == 'r':
            new_state = self.move(x, y, x, y + 1)
        elif movement == 'l':
            new_state = self.move(x, y, x, y - 1)
        elif movement == 'u':
            new_state = self.move(x, y, x - 1, y)
        elif movement == 'd':
            new_state = self.move(x, y, x + 1, y)
        else:
            return None

        if new_state is not None:
            # create child node, increment g by 1, set parent to current node
            return Node(new_state, g=self.g + 1, parent=self)
        return None

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __repr__(self):
        return f"Node(f={self.f}, g={self.g}, h={self.h})"


class Puzzle:
    def __init__(self):
        self.open = []          # nodes to explore
        self.closed = []        # already explored nodes
        self.goal_state = None

    def read_input(self, filename="input.txt"):
        with open(filename, 'r') as file:
            lines = file.readlines()
            lines = [line.strip().split(" ") for line in lines]

        start_state = lines[1:4]
        goal_state = lines[5:8]
        self.goal_state = goal_state
        return start_state, goal_state


    def init_start_node(self, start_state, goal_state):
        g_s = 0
        h_s = manhattan_distance(start_state, goal_state)
        start = Node(start_state, g=g_s, h=h_s)
        self.open.append(start)

        print("Placed start node into OPEN.")
        self.display_node(start, "Start Node")


    def is_open_empty(self):
        if not self.open:
            print("Failure: OPEN list is empty. No solution exists.")
            return True
        return False
    

    def get_best_node(self):
        n = min(self.open, key=lambda node: node.f)
        self.open.remove(n)
        self.closed.append(n)
        print("\nSelected node with smallest f from OPEN:")
        self.display_node(n, "n")
        print(f"Moved n from OPEN to CLOSED.")
        print(f"OPEN size: {len(self.open)}, CLOSED size: {len(self.closed)}\n")
        return n


    def display_node(self, node, name="Node"):
        print(f"{name}:")
        for row in node.state:
            print(" ".join(row))
        print(f"g = {node.g}, h = {node.h}, f = {node.f}\n")

    def is_goal(self, node):
        return node.state == self.goal_state
    
    def reconstruct_path(self, node):
        path = []
        current = node

        while current:
            path.append(current)
            current = current.parent
        
        path.reverse()
        return path

    def display_solution(self, path):
        print("\nSolution Found!")
        print(f"Number of moves: {len(path) - 1}\n")

        for step, node in enumerate(path):
            print(f"Step {step} (g={node.g}, h={node.h}, f={node.f}):")
            for row in node.state:
                print(" ".join(row))
            print()
    
    def expand_node(self, node):
        children = []
        directions = ['r', 'l', 'u', 'd']

        for move in directions:
            child = node.generate(move)
            if child is not None:
                # compute heuristic for child
                child.h = manhattan_distance(child.state, self.goal_state)
                child.f = child.g + child.h
                children.append(child)
        return children
    
    def add_children_to_open(self, children, parent):
        for child in children:
            #Searching existing node in OPEN and CLOSED
            open_match = next((c for c in self.open if c.state == child.state), None)
            closed_match = next((c for c in self.closed if c.state == child.state), None)

            if open_match is None and closed_match is None:
                # add the new nodes to OPEN
                self.open.append(child)
                child.parent = parent
                print("Added child to OPEN:")
                self.display_node(child)
            else:
                # it already exists in OPEN or CLOSED
                existing = open_match if open_match else closed_match

                if child.f < existing.f:
                    print("Found a better path to an existing node:")
                    self.display_node(child)

                    # update existing node with better value
                    existing.g = child.g
                    existing.h = child.h
                    existing.f = child.f
                    existing.parent = parent

                    if closed_match:
                        # move it back to OPEN
                        self.closed.remove(existing)
                        self.open.append(existing)
                        print("Moved node from CLOSED back to OPEN with updated f")

    def start(self):
        start_state, goal_state = self.read_input("input.txt")

        # STEP 1: Initialize start node and put it in OPEN
        self.init_start_node(start_state, goal_state)

        #STEP 8: Loop until solution is found or failure
        while True:
            # STEP 2: Check if OPEN is empty
            if self.is_open_empty():
                return

            # STEP 3: Get node with smallest f from OPEN and move it to CLOSED
            n = self.get_best_node()

            # STEP 4: Check if n is the goal state
            if self.is_goal(n):
                path = self.reconstruct_path(n)
                self.display_solution(path)
                return
            else:
                print("n is not the goal state. Generating children...\n")

            # STEP 5: Expand node n, generate all children
            children = self.expand_node(n)

            if not children:
                print("No successors for this node. Returning to Step 2...\n")
                return
            else:
                print(f"Generated {len(children)} successors of n:\n")
                for idx, child in enumerate(children):
                    print(f"Child {idx + 1}:")
                    for row in child.state:
                        print(" ".join(row))
                    print(f"g = {child.g}, h = {child.h}, f = {child.f}\n")
        
            #STEP 6 and 7: Insert children or update existing nodes
            self.add_children_to_open(children, n)


if __name__ == "__main__":
    p = Puzzle()
    p.start()

Placed start node into OPEN.
Start Node:
2 1 6
4 * 8
7 5 3
g = 0, h = 12, f = 12


Selected node with smallest f from OPEN:
n:
2 1 6
4 * 8
7 5 3
g = 0, h = 12, f = 12

Moved n from OPEN to CLOSED.
OPEN size: 0, CLOSED size: 1

n is not the goal state. Generating children...

Generated 4 successors of n:

Child 1:
2 1 6
4 8 *
7 5 3
g = 1, h = 11, f = 12

Child 2:
2 1 6
* 4 8
7 5 3
g = 1, h = 11, f = 12

Child 3:
2 * 6
4 1 8
7 5 3
g = 1, h = 13, f = 14

Child 4:
2 1 6
4 5 8
7 * 3
g = 1, h = 13, f = 14

Added child to OPEN:
Node:
2 1 6
4 8 *
7 5 3
g = 1, h = 11, f = 12

Added child to OPEN:
Node:
2 1 6
* 4 8
7 5 3
g = 1, h = 11, f = 12

Added child to OPEN:
Node:
2 * 6
4 1 8
7 5 3
g = 1, h = 13, f = 14

Added child to OPEN:
Node:
2 1 6
4 5 8
7 * 3
g = 1, h = 13, f = 14


Selected node with smallest f from OPEN:
n:
2 1 6
4 8 *
7 5 3
g = 1, h = 11, f = 12

Moved n from OPEN to CLOSED.
OPEN size: 3, CLOSED size: 2

n is not the goal state. Generating children...

Generated 3 successors of n: